In [8]:
!pip install PyPDF2 langchain langchain-community ollama langchain-groq

In [18]:
!pip install --upgrade langchain

In [10]:
!pip install pinecone

In [11]:
!pip install groq

In [16]:
!pip install ollama

In [19]:
!pip install langchain-community

In [22]:
!pip install -U langchain-ollama

In [24]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [31]:
import os
os.environ["PINECONE_API_KEY"] = "3a636cf5-1f85-45a4-b7b4-6bf7b0a79b23"
os.environ["GROQ_API_KEY"] = "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"
os.environ["COHERE_API_KEY"] = "CuheiKs3XVjEJ3GZhz4JdjjbpRwuSKSSbWa1QiLK"

In [32]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


llm_local = ChatOllama(model="mistral:instruct")
llm_groq = ChatGroq(
            groq_api_key=groq_api_key,
            #model_name='llama2-70b-4096' 
            model_name='mixtral-8x7b-32768'
    )

In [34]:
# Read the PDF file
pdf = PyPDF2.PdfReader("Brain And Learning.pdf")
pdf_text = ""
for page in pdf.pages:
    pdf_text += page.extract_text()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(pdf_text)

embeddings = OllamaEmbeddings(model="nomic-embed-text")

r1 = embeddings.embed_documents(
    texts
)

In [35]:

import pinecone
from pinecone import Pinecone, ServerlessSpec
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

pc.create_index(
  name="testv1",
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

index = pc.Index("testv1")

for i in range(len(texts)):
    index.upsert([((str(i),r1[i],{"text":texts[i]}))])
    
print("done upserting...")

def get_query_embdedding(text):
    embedding=embeddings.embed_query(text)
    return embedding



done upserting...


In [37]:
!pip install cohere

  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached parameterized-0.9.0-py2.py3-none-any.whl.metadata (18 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached parameterized-0.9.0-py2.py3-none-any.whl (20 kB)


In [38]:
import cohere
load_dotenv()
cohere_api_key = os.getenv("COHERE_API_KEY")
 
# init client
co = cohere.Client(cohere_api_key)

query="The facilitative role of enrichment in learning?"

question_embedding=get_query_embdedding(query)

query_result = index.query(vector=question_embedding, top_k=5, include_metadata=True)
similar_texts = []
# Extract metadata from query result
docs = {x["metadata"]['text']: i for i, x in enumerate(query_result["matches"])}



# Rerank the documents
rerank_docs = co.rerank(
    model="rerank-english-v3.0",
    query=query, 
    documents=list(docs.keys()), 
    top_n=5, 
    return_documents=True
)

In [39]:

# Extract reranked documents
reranked_texts = [doc.document.text for doc in rerank_docs.results]
reranked_texts

context=" ".join(reranked_texts)

Template = f"Based on the following context : {context} generate precise summary related to question : {query} Do not remove necessary information related to context. Consider `\n` as newline character."  
# Filling the template with the actual context and question.
filled_template = Template.format(context=context, question=query)

In [40]:

import os
from groq import Groq

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

client = Groq(
    api_key=groq_api_key ,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": filled_template,
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

The facilitative role of enrichment in learning is centered on the idea that integrating complementary sensory and motor information into the learning experience can enhance it. Enriched educational methods have been found to be effective because they engage greater levels of attention and deeper levels of processing, as well as because multimodal interactions in the brain can enhance learning and memory.

Recent advances in neuroscience research show that sensory and motor areas in the brain respond crossmodally to enriched items following learning, and these brain responses occur during early stages of perceptual processing. These crossmodal brain responses have been found to contribute causally to the behavioral benefits of enrichment, supporting multimodal theories of enrichment.

The translation of neuroscience-based principles to classroom experiments has shown promising results, but more rigorous research is needed to understand how enrichment can best be used to support learnin